In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import itertools
import os

%matplotlib inline
from finrl import config
from finrl import config_tickers
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

/home/henry/miniconda3/envs/finrl/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [2]:
import datetime


In [3]:
from data_processor.indicator import (
    add_all_indicators,
    add_tech_indicator,
    tech_indicator_only,
)


In [4]:
import data_processor.config


In [6]:
price_df = pd.read_parquet("./data/sz50_price.parquet")
fund_df = pd.read_parquet("./data/sz50_fundament.parquet")


In [7]:
df = add_all_indicators(price_df, fund_df)
#df = add_tech_indicator(price_df)
df.head()


,date,tic,open,high,low,close,volume,amount,peTTM,pbMRQ,...,macdhist,CCI,RSI,NATR,ADOSC,ROE,AssetStoEquity,Pnitoni,Nitogr,TaxBurden
0,2008-06-30,sh.600000,47.588843,48.725921,45.798997,46.325422,18472630.0,4.119299e+08,16.993591,3.964146,...,-0.312665,-249.983045,33.922854,6.516124,-2.366369e+06,0.094165,31.230895,1.000000,0.349795,0.761823
1,2008-06-30,sh.600028,11.591980,12.009291,11.336956,11.765860,66219882.0,6.663410e+08,20.920215,2.797929,...,-0.274170,-165.029880,35.553363,8.545361,1.887818e+07,0.021775,2.424673,1.008579,0.020011,0.972198
2,2008-06-30,sh.600030,50.107056,52.194850,49.104915,49.940032,62758174.0,1.514636e+09,11.614174,3.039415,...,-0.491795,-92.445524,35.494104,8.747922,4.709105e+07,0.048595,3.940038,0.912936,0.476801,0.773185
3,2008-06-30,sh.600031,115.910200,119.760075,111.811947,115.123666,5576465.0,1.553746e+08,17.886464,5.505186,...,0.392326,-91.666667,31.941251,7.312560,1.320480e+06,0.062107,2.457262,0.858524,0.132264,0.875612
4,2008-06-30,sh.600036,48.239143,49.304624,48.051117,48.928572,41156575.0,9.584711e+08,18.033613,4.607180,...,0.295525,-135.510793,34.556248,5.260317,1.371192e+07,0.088535,18.804996,1.000000,0.460837,0.760135


In [8]:
available_tics = (
    df.set_index("date").loc[data_processor.config.FIRST_DAY_OF_2009].tic.tolist()
)


In [9]:
tmp_list = []
for tic in available_tics:
    tmp_df = df[df['tic'] == tic]
    tmp_list.append(tmp_df)
    
df = pd.concat(tmp_list, ignore_index=True)
    

In [10]:
train = data_split(df, "2009-01-01", "2020-01-01")
trade = data_split(df, "2020-01-01", "2022-01-01")


In [11]:
indicators = data_processor.config.INDICATORS
# indicators = ["SMA_20", "SMA_60", "SMA_120", "macdhist", "CCI", "RSI", "NATR", "ADOSC"]


In [12]:
n_indicators = len(indicators)
n_indicators


17

In [13]:
stock_dimension = len(df.tic.unique())
stock_dimension


34

In [14]:
state_space = 1 + (2 + n_indicators) * stock_dimension
state_space


647

In [15]:
env_kwargs = {
    "hmax": 1000,
    "initial_amount": 10000000,
    # "initial_list": [10000000] + [0 for i in range(stock_dimension)],
    # buy and sell cost for each stock
    "num_stock_shares": [0] * stock_dimension,
    "buy_cost_pct": [0.001] * stock_dimension,
    "sell_cost_pct": [0.001] * stock_dimension,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": indicators,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
}

e_train_gym = StockTradingEnv(df=train, **env_kwargs)


In [16]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))


<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


### SAC

In [17]:
agent = DRLAgent(env=env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model(
    "sac", model_kwargs=SAC_PARAMS, tensorboard_log=config.TENSORBOARD_LOG_DIR
)


{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cuda device


In [18]:
trained_sac = agent.train_model(
    model=model_sac, tb_log_name="sac", total_timesteps=10000
)


Logging to tensorboard_log/sac_3


In [19]:
model = trained_sac

### DDPG

In [22]:
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 100000, "learning_rate": 0.0001}

agent = DRLAgent(env=env_train)
# model_ddpg = agent.get_model(
#     "ddpg", model_kwargs=DDPG_PARAMS, tensorboard_log=config.TENSORBOARD_LOG_DIR
# )
model_ddpg = agent.get_model("ddpg", model_kwargs=DDPG_PARAMS)


{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001}
Using cuda device


In [23]:
trained_ddpg = agent.train_model(
    model=model_ddpg, tb_log_name="ddpg", total_timesteps=1
)


In [24]:
model = trained_ddpg

### PPO

In [ ]:
agent = DRLAgent(env=env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model(
    "ppo", model_kwargs=PPO_PARAMS, tensorboard_log=config.TENSORBOARD_LOG_DIR
)


In [ ]:
trained_ppo = agent.train_model(
    model=model_ppo, tb_log_name="ppo", total_timesteps=100000
)

model = trained_ppo


## Backtest

In [20]:
e_trade_gym = StockTradingEnv(df=trade, **env_kwargs)


In [21]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=model, environment=e_trade_gym
)


hit end!


In [25]:
df_account_value.iloc[-1].account_value


18330115.182306215

In [25]:
# df_account_value.to_csv("./sz50_without_fund.csv")


In [26]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime("%Y%m%d-%Hh%M")

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)


==============Get Backtest Results===========


In [27]:
from data_processor.ChinaStockDownloader import single_stock_query

baseline_df = single_stock_query("2020-01-01", "2022-01-01", "sh.000016")


login success!
logout success!


In [28]:
# baseline stats
print("==============Get Baseline Stats===========")
baseline_df = single_stock_query("2020-01-01", "2022-01-01", "sh.000016")
baseline_df = baseline_df.apply(pd.to_numeric, errors="ignore")
stats = backtest_stats(baseline_df, value_col_name="close")


==============Get Baseline Stats===========
login success!
logout success!


In [ ]:
test_ret = get_daily_return(df_account_value)
base_ret = get_daily_return(baseline_df, "close")


In [ ]:
print("==============Compare to SZ50===========")
%matplotlib inline

import pyfolio

In [ ]:
with_fund = pd.read_csv("./sz50_with_fund.csv")
without_fund = pd.read_csv("./sz50_without_fund.csv")
with_fund_ret = get_daily_return(with_fund)
without_fund_ret = get_daily_return(without_fund)
with_fund_ret.index = pd.DatetimeIndex(with_fund_ret.index.date)
without_fund_ret.index = pd.DatetimeIndex(without_fund_ret.index.date)

In [ ]:
test_ret.index = pd.DatetimeIndex(test_ret.index.date)
base_ret.index = pd.DatetimeIndex(base_ret.index.date)


In [ ]:
# import quantstats as qs

# qs.reports.html(with_fund_ret, without_fund_ret)


In [ ]:
pyfolio.create_full_tear_sheet(returns=test_ret, benchmark_rets=base_ret)
